In [32]:
import psycopg2
import pandas as pd
import datetime
import numpy as np
from unidecode import unidecode
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import IsolationForest
import os
import sys
import pickle


In [2]:
import shap

ModuleNotFoundError: No module named 'shap'

## FEATURE ENGINEERING EXP

In [33]:
# codificaciones = pd.read_excel(
#     r"cce/src/Data/Codificaciones.xlsx", sheet_name="codificaciones"
# )
codificaciones = pd.read_excel(
    r"/home/cnvdba/cce/src_dev/Data/Codificaciones.xlsx", sheet_name="codificaciones"
)
codificaciones.Code = codificaciones.Code.str.strip()
rename_columns = {
    "creation_date": "creationDate",
    "creation_time": "creationTime",
    "creditor_participant_code": "creditorParticipantCode",
    "debtor_participant_code": "debtorParticipantCode",
    "debtor_type_person": "debtorTypeOfPerson",
    "transaction_type": "transactionType",
    "debtor_id": "debtorId",
    "debtor_id_code": "debtorIdCode",
    "creditor_cci": "creditorCCI",
    "creditor_credit_card": "creditorCreditCard",
    "reason_code": "reasonCode",
    "response_code": "responseCode",
    "creditor_id": "creditorId",
    "creditor_id_code": "creditorIdCode",
}

dict_column_list = {
    "debtorParticipantCode": "participants",
    "creditorParticipantCode": "participants",
    "transactionType": "transaction_type",
    "currency": "currency",
    "channel": "channel",
    "responseCode": "response_code",
    "reasonCode": "reason_code",
}

In [34]:
_driver = "PostgreSQL ANSI(x64)"
_server = "localhost" #"10.201.4.25" 
_database = "coemcas"
_username = "coemcas"
_password = "C03$CMa5$2099"
_port = "5432"
PREDICTION_PIPELINE = True

def get_database_conection():
    connection = psycopg2.connect(
        host=_server,
        database=_database,
        user=_username,
        password=_password,
        port=_port
    )
    return connection
# 2024-08-15	2024-08-15 12:25:00.000	2024-08-15 12:29:59.000

In [35]:
def data_ingestion_from_database():
    #TODO: REVERT SQL QUERY
    #Given the first unrun batch it finds, it retrieves all the transactions from the last 7 days (excluding the unrun batch) 
    if PREDICTION_PIPELINE:
        # sql = """
        #     with 

        #     run_id_to_process as (
        #     select r.run_id, r.run_date 
        #     from fraud_model_run r 
        #     where r.run_start_datetime = (select min(r2.run_start_datetime) from fraud_model_run r2 where coalesce(r2.run_process_status,0) = 0)),

        #     debtor_ids as (
        #     select distinct m.debtor_id, r.run_id as current_run_id, r.run_date as current_run_date
        #     from run_id_to_process r
        #     inner join stage_ipf_message m on r.run_id = m.run_id)

            # select m.*
            # from stage_ipf_message m 
            # inner join debtor_ids d on m.debtor_id = d.debtor_id
            # inner join fraud_model_run r on m.run_id = r.run_id
            # where r.run_id <= d.current_run_id and r.run_date >= current_run_date - INTERVAL '7 days'"""
        #TODO: temporarily remove duplicate rows (by pk)
        # sql = """
        #     WITH run_id_to_process AS (
        #         SELECT r.run_id, r.run_date 
        #         FROM fraud_model_run r 
        #         WHERE r.run_start_datetime = (
        #             SELECT MIN(r2.run_start_datetime) 
        #             FROM fraud_model_run r2 
        #             WHERE COALESCE(r2.run_process_status, 0) = 0
        #         )
        #     ),
        #     debtor_ids AS (
        #         SELECT DISTINCT m.debtor_id, r.run_id AS current_run_id, r.run_date AS current_run_date
        #         FROM run_id_to_process r
        #         INNER JOIN stage_ipf_message m ON r.run_id = m.run_id
        #     )
        #     SELECT DISTINCT ON (m.pk) m.*
        #     FROM stage_ipf_message m 
        #     INNER JOIN debtor_ids d ON m.debtor_id = d.debtor_id
        #     --INNER JOIN fraud_model_run r ON m.run_id = r.run_id
        #     --WHERE r.run_id <= d.current_run_id 
        #     --AND r.run_date >= current_run_date - INTERVAL '90 days';
        #     WHERE m.creation_date BETWEEN d.current_run_date - interval '90 days' AND d.current_run_date;
        #     """
        #PENULTIMO
        # sql = """
        #     WITH run_id_to_process AS (
        #         SELECT r.run_id, r.run_date, r.run_end_datetime
        #         FROM fraud_model_run r 
        #         WHERE r.run_start_datetime = (
        #                 SELECT MIN(r2.run_start_datetime) 
        #                 FROM fraud_model_run r2 
        #                 WHERE COALESCE(r2.run_process_status, 0) = 0
        #         )
        #     ),
        #     debtor_ids AS (
        #         SELECT DISTINCT m.debtor_id, r.run_id AS current_run_id, r.run_date AS current_run_date, r.run_end_datetime
        #         FROM run_id_to_process r
        #         INNER JOIN stage_ipf_message m ON r.run_id = m.run_id
        #     )
        #     SELECT DISTINCT ON (m.pk) m.*, d.current_run_id
        #     FROM stage_ipf_message m 
        #     INNER JOIN debtor_ids d ON m.debtor_id = d.debtor_id
        #     WHERE m.creation_date BETWEEN d.current_run_date - interval '90 days' AND d.current_run_date
        #     AND (
        #         m.creation_date < d.current_run_date 
        #         OR (m.creation_date = d.current_run_date AND m.creation_time <= d.run_end_datetime::time)
        #     );
        # """
        #ULTIMO
        sql = """
            WITH run_id_to_process AS (
                 SELECT r.run_id, r.run_date, r.run_end_datetime
                FROM fraud_model_run r 
                WHERE r.run_id = '20240828_0057'
            ),
            debtor_ids AS (
                SELECT DISTINCT m.debtor_id, r.run_id AS current_run_id, r.run_date AS current_run_date, r.run_end_datetime
                FROM run_id_to_process r
                INNER JOIN stage_ipf_message m ON r.run_id = m.run_id
            )
            SELECT DISTINCT ON (m.pk) m.*, d.current_run_id
            FROM stage_ipf_message m 
            INNER JOIN debtor_ids d ON m.debtor_id = d.debtor_id
            WHERE m.run_id = d.current_run_id
            OR (
                m.run_id != d.current_run_id
                AND m.creation_date BETWEEN d.current_run_date - interval '90 days' AND d.current_run_date
                AND (
                    m.creation_date < d.current_run_date 
                    OR (m.creation_date = d.current_run_date AND m.creation_time <= d.run_end_datetime::time)
                )
            );

        """
    else:
        # sql = """
        #     with run_id_to_process as (
        #         select distinct creation_date 
        #         from stage_ipf_message_old 
        #         order by creation_date desc 
        #         limit 1
        #     ),
        #     debtor_ids as (
        #     select distinct m.debtor_id, m.creation_date as current_creation_date
        #     from stage_ipf_message_old m
        #     where m.creation_date >= '2023-05-19') -- in (select creation_date from run_id_to_process))

        #     select m.*
        #     from stage_ipf_message_old m
        #     inner join debtor_ids d on m.debtor_id = d.debtor_id
        #     where m.creation_date >= d.current_creation_date - interval '90 days' 
        # """
        sql = """
            with debtor_ids as (
                select debtor_id, current_creation_date
                from (
                    select 
                        debtor_id, 
                        current_creation_date,
                        row_number() over (partition by current_creation_date order by random()) as rn
                    from (
                        select 
                            distinct m.debtor_id, 
                            m.creation_date as current_creation_date
                        from stage_ipf_message_old m
                        where m.creation_date between '2022-08-30' and '2022-09-02'
                    ) distinct_pairs
                ) numbered_pairs
                where rn <= 200000
            ),
            filtered_rows as (
                select 
                    m.*, 
                    row_number() over (partition by m.pk) as row_num
                from stage_ipf_message_old m
                inner join debtor_ids d on m.debtor_id = d.debtor_id
                where m.creation_date between d.current_creation_date - interval '7 days' and d.current_creation_date
            )
            select *
            from filtered_rows
            where row_num = 1;
        """
        #     sql = """with 

        # run_id_to_process as (
        # select r.run_id, r.run_date 
        # from fraud_model_run r 
        # where r.run_id = '20240410_0168'),

        # debtor_ids as (
        # select distinct m.debtor_id, r.run_id as current_run_id, r.run_date as current_run_date
        # from run_id_to_process r
        # inner join stage_ipf_message m on r.run_id = m.run_id)

        # select m.*
        # from stage_ipf_message m 
        # inner join debtor_ids d on m.debtor_id = d.debtor_id
        # inner join fraud_model_run r on m.run_id = r.run_id
        # where r.run_id <= d.current_run_id and r.run_date >= current_run_date - INTERVAL '7 days'"""

    connection = None
    # Establish the database connection
    connection = get_database_conection()
    cursor = connection.cursor()
    
    print(f"EXEC - Started query to get AV_consolidado rows at {datetime.datetime.now()}")
    cursor.execute(sql)
    print(f"EXEC - Ended query (cursor.execute) to get AV_consolidado rows at {datetime.datetime.now()}")
    # Obtener los nombres de las columnas
    column_names = [desc[0] for desc in cursor.description]
    # Obtener los resultados como una lista de listas
    results = cursor.fetchall()
    print(f"EXEC - Ended query (cursor.fetchall) to get AV_consolidado rows at {datetime.datetime.now()}")
    cursor.close()
    if connection is not None:
        connection.close()
    # Convertir los resultados en un DataFrame
    AV_consolidado = pd.DataFrame(results, columns=column_names)
    print(column_names)
    return AV_consolidado

In [36]:
AV_consolidado = data_ingestion_from_database()

EXEC - Started query to get AV_consolidado rows at 2024-09-05 10:32:21.205434
EXEC - Ended query (cursor.execute) to get AV_consolidado rows at 2024-09-05 10:32:29.785886
EXEC - Ended query (cursor.fetchall) to get AV_consolidado rows at 2024-09-05 10:32:32.475598
['pk', 'debtor_id', 'creditor_cci', 'creation_date', 'creation_time', 'channel', 'currency', 'creditor_participant_code', 'debtor_participant_code', 'debtor_type_person', 'transaction_type', 'debtor_id_code', 'creditor_credit_card', 'reason_code', 'response_code', 'creditor_id', 'creditor_id_code', 'message_id', 'trace', 'instruction_id', 'run_id', 'last_modified', 'log_timestamp_replica', 'same_customer_flag', 'current_run_id']


In [37]:
if PREDICTION_PIPELINE:
    max_run_id = AV_consolidado.iloc[0]["current_run_id"]
else:
    max_run_id = "XXX"
drop_cols_av_cons = (
    ['log_timestamp_replica', 'last_modified', 'current_run_id', 'same_customer_flag'] 
    if PREDICTION_PIPELINE 
    else ['last_modified']
)
AV_consolidado = AV_consolidado.drop(columns=drop_cols_av_cons)

In [38]:
max_run_id

'20240904_0101'

In [39]:
AV_consolidado.shape

(646844, 21)

In [40]:
import time
# TODO: lentisimo

# AV_consolidado=AV_consolidado.rename(columns=rename_columns)
# AV_consolidado_original_stage = AV_consolidado.copy()
# AV_consolidado_original = pd.DataFrame()
# AV_consolidado_original_values_set = pd.DataFrame()
# AV_consolidado_completed = pd.DataFrame()
# for key, value in dict_column_list.items():
#     column_to_decode = key
#     column_to_decode_index = AV_consolidado.columns.get_loc(column_to_decode)

#     codificaciones_stage = codificaciones[codificaciones.List == value]  # "participants"]
#     AV_consolidado[column_to_decode] = AV_consolidado[column_to_decode].fillna("")
#     AV_consolidado_stage = pd.merge(AV_consolidado,codificaciones_stage[["Code", "Value"]],left_on=column_to_decode,right_on="Code",how="left",)

#     if column_to_decode not in ["reasonCode"]:
#         # column_data = AV_consolidado_stage["Value"].fillna("invalid_"+AV_consolidado_stage[column_to_decode])
#         column_data = AV_consolidado_stage["Value"].fillna("invalid")
#     else:
#         column_data = AV_consolidado_stage["Value"]

#     AV_consolidado_stage.drop(["Value", "Code", column_to_decode], axis=1, inplace=True)
#     AV_consolidado_stage.insert(column_to_decode_index, column_to_decode, column_data)
#     AV_consolidado = AV_consolidado_stage.copy()
# AV_consolidado = AV_consolidado.rename(
#     columns={"debtorParticipantCode": "debtorParticipant",
#         "creditorParticipantCode": "creditorParticipant",
#     }
# )

# ALTERNATIVA
# Renombrar columnas in-place
AV_consolidado.rename(columns=rename_columns, inplace=True)
# Recorrer el diccionario para procesar cada columna
# print(dict_column_list)
# start = time.time()
# for column_to_decode, value in dict_column_list.items():
#     print(column_to_decode)
#     # Filtrar codificaciones relevantes
#     codificaciones_stage = codificaciones[codificaciones['List'] == value][["Code", "Value"]]

#     # Rellenar valores nulos en la columna a decodificar
#     AV_consolidado[column_to_decode].fillna("", inplace=True)

#     # Realizar el merge de forma más eficiente
#     AV_consolidado = pd.merge(
#         AV_consolidado,
#         codificaciones_stage[['Code', 'Value']],
#         left_on=column_to_decode,
#         right_on='Code',
#         how='left'
#     )

#     # Asignar valores decodificados o marcar como 'invalid' in-place
#     if column_to_decode == "reasonCode":
#         AV_consolidado[column_to_decode] = AV_consolidado['Value']
#     else:
#         AV_consolidado[column_to_decode] = AV_consolidado['Value'].fillna("invalid")

#     # Eliminar columnas auxiliares para liberar memoria
#     AV_consolidado.drop(['Value', 'Code'], axis=1, inplace=True)
####NUEVA
start = time.time()
for column_to_decode, value in dict_column_list.items():
    print(column_to_decode)
    # Filtrar codificaciones relevantes
    codificaciones_stage = codificaciones[codificaciones['List'] == value][["Code", "Value"]]
    codificaciones_dict = codificaciones_stage.set_index('Code')['Value'].to_dict()

    # Rellenar valores nulos en la columna a decodificar
    AV_consolidado[column_to_decode].fillna("", inplace=True)

    # Realizar el merge de forma más eficiente
    AV_consolidado['temp'] = AV_consolidado[column_to_decode].map(codificaciones_dict)

    if column_to_decode not in ["reasonCode"]:
        AV_consolidado['temp'] = AV_consolidado['temp'].fillna("invalid")
 
    # Insertar la columna mapeada en el lugar adecuado
    AV_consolidado[column_to_decode] = AV_consolidado['temp']
    AV_consolidado.drop('temp', axis=1, inplace=True)
# Renombrar columnas finales in-place
AV_consolidado.rename(
    columns={
        "debtorParticipantCode": "debtorParticipant",
        "creditorParticipantCode": "creditorParticipant",
    },
    inplace=True
)
end = time.time()
print("TOT TIME", end - start)

debtorParticipantCode


/tmp/ipykernel_937019/2169875756.py:72: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  AV_consolidado[column_to_decode].fillna("", inplace=True)


creditorParticipantCode


/tmp/ipykernel_937019/2169875756.py:72: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  AV_consolidado[column_to_decode].fillna("", inplace=True)


transactionType


...

responseCode


/tmp/ipykernel_937019/2169875756.py:72: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  AV_consolidado[column_to_decode].fillna("", inplace=True)


reasonCode


/tmp/ipykernel_937019/2169875756.py:72: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  AV_consolidado[column_to_decode].fillna("", inplace=True)


TOT TIME 4.369802951812744


In [41]:
AV_consolidado.creationDate.drop_duplicates().sort_values().reset_index(drop=True)

0     2024-06-06
1     2024-06-07
2     2024-06-08
3     2024-06-09
4     2024-06-10
         ...    
86    2024-08-31
87    2024-09-01
88    2024-09-02
89    2024-09-03
90    2024-09-04
Name: creationDate, Length: 91, dtype: object

In [42]:
AV_consolidado["flag_invalid"] = (
    AV_consolidado.astype(str).apply(lambda x: x == "invalid").any(axis=1)
).astype("uint8")

AV_consolidado_original_values_set_stage = AV_consolidado.copy()

In [43]:
AV_consolidado.creationDate.drop_duplicates().sort_values().reset_index(drop=True)

0     2024-06-06
1     2024-06-07
2     2024-06-08
3     2024-06-09
4     2024-06-10
         ...    
86    2024-08-31
87    2024-09-01
88    2024-09-02
89    2024-09-03
90    2024-09-04
Name: creationDate, Length: 91, dtype: object

In [44]:
AV_consolidado.reset_index(drop=True, inplace=True)
rows_dropped = (
    AV_consolidado_original_values_set_stage.shape[0] - AV_consolidado.shape[0]
)
if rows_dropped > 0:
    print(f"{rows_dropped} rows dropped because any value was invalid")
AV_consolidado.drop(["flag_invalid"], axis=1, inplace=True)

In [45]:
AV_consolidado.creationDate.drop_duplicates().sort_values().reset_index(drop=True)

0     2024-06-06
1     2024-06-07
2     2024-06-08
3     2024-06-09
4     2024-06-10
         ...    
86    2024-08-31
87    2024-09-01
88    2024-09-02
89    2024-09-03
90    2024-09-04
Name: creationDate, Length: 91, dtype: object

In [46]:
# vars_to_discard = [
#     "pk",
#     "reasonCode",
#     "run_id",
#     "trace",
#     "instruction_id",
#     "message_id",
# ]  # 1
# vars_to_feature_engineer = ["creationDate", "creationTime"]  # 2
# vars_to_ohe = [
#     "debtorTypeOfPerson",
#     "debtorParticipant",
#     "creditorParticipant",
#     "transactionType",
#     "currency",
#     "channel",
#     "responseCode",
# ]  # 3

# # 1 vars_to_discard
# AV_consolidado.drop(vars_to_discard, axis=1, inplace=True)

# # 2 vars_to_feature_engineer
# AV_consolidado.rename(
#     columns={
#         "creationDate": "creationDate_stage",
#         "creationTime": "creationTime_stage",
#     },
#     inplace=True,
# )

# AV_consolidado["creationDate"] = pd.to_datetime(
#     AV_consolidado["creationDate_stage"]
# ).dt.strftime("%Y%m%d")
# AV_consolidado["creationTime_stage"] = (
#     AV_consolidado["creationTime_stage"].astype("string").str.replace(":", "")
# )
# AV_consolidado["creationTime"] = pd.to_datetime(
#     AV_consolidado["creationTime_stage"], format="%H%M%S"
# ).dt.time

# AV_consolidado.drop(
#     ["creationDate_stage", "creationTime_stage"], axis=1, inplace=True
# )
# AV_consolidado["hourSin"] = AV_consolidado["creationTime"].apply(
#     lambda x: np.sin(2 * np.pi * x.hour / 24.0)
# )
# AV_consolidado["hourCos"] = AV_consolidado["creationTime"].apply(
#     lambda x: np.cos(2 * np.pi * x.hour / 24.0)
# )
# AV_consolidado["creationDate"] = pd.to_datetime(AV_consolidado["creationDate"])

# AV_consolidado["dayOfYearSin"] = AV_consolidado["creationDate"].apply(
#     lambda x: np.sin(2 * np.pi * x.timetuple().tm_yday / 365.0)
# )
# AV_consolidado["dayOfYearCos"] = AV_consolidado["creationDate"].apply(
#     lambda x: np.cos(2 * np.pi * x.timetuple().tm_yday / 365.0)
# )
# AV_consolidado["dayOfMonthSin"] = AV_consolidado["creationDate"].apply(
#     lambda x: np.sin(2 * np.pi * x.day / 31.0)
# )
# AV_consolidado["dayOfMonthCos"] = AV_consolidado["creationDate"].apply(
#     lambda x: np.cos(2 * np.pi * x.day / 31.0)
# )
# AV_consolidado["dayOfWeekSin"] = AV_consolidado["creationDate"].apply(
#     lambda x: np.sin(2 * np.pi * x.weekday() / 7.0)
# )
# AV_consolidado["dayOfWeekCos"] = AV_consolidado["creationDate"].apply(
#     lambda x: np.cos(2 * np.pi * x.weekday() / 7.0)
# )
# AV_consolidado["monthSin"] = AV_consolidado["creationDate"].apply(
#     lambda x: np.sin(2 * np.pi * x.month / 12.0)
# )
# AV_consolidado["monthCos"] = AV_consolidado["creationDate"].apply(
#     lambda x: np.cos(2 * np.pi * x.month / 12.0)
# )

# AV_consolidado.drop(vars_to_feature_engineer, axis=1, inplace=True)

# # 3 vars_to_ohe
# AV_consolidado_3_stage = AV_consolidado.copy()

# # Cambiar formato a "snake_case" sin tildes
# for column in vars_to_ohe:
#     AV_consolidado_3_stage[column] = (
#         AV_consolidado_3_stage[column]
#         .astype(str)
#         .apply(lambda x: unidecode(x).replace(" ", "_").lower())
#     )

In [47]:
vars_to_discard = [
    "pk",
    "reasonCode",
    "run_id",
    "trace",
    "instruction_id",
    "message_id",
]  # 1
vars_to_feature_engineer = ["creationDate", "creationTime"]  # 2
vars_to_ohe = [
    "debtorTypeOfPerson",
    "debtorParticipant",
    "creditorParticipant",
    "transactionType",
    "currency",
    "channel",
    "responseCode",
]  # 3


In [48]:
########NEW
# 1 vars_to_discard
AV_consolidado = AV_consolidado.drop(columns=vars_to_discard)

# 2 vars_to_feature_engineer
AV_consolidado = AV_consolidado.rename(
    columns={
        "creationDate": "creationDate_stage",
        "creationTime": "creationTime_stage",
    }
)

# Conversión de fechas y horas
AV_consolidado["creationDate_stage"] = pd.to_datetime(AV_consolidado["creationDate_stage"])
AV_consolidado["creationTime_stage"] = AV_consolidado["creationTime_stage"].astype(str).str.replace(":", "", regex=False)
AV_consolidado["creationTime"] = pd.to_datetime(AV_consolidado["creationTime_stage"], format="%H%M%S").dt.time

# Creación de características cíclicas
def create_cyclic_features(df):
    df["hourSin"] = np.sin(2 * np.pi * df["creationTime"].apply(lambda x: x.hour) / 24.0)
    df["hourCos"] = np.cos(2 * np.pi * df["creationTime"].apply(lambda x: x.hour) / 24.0)
    df["dayOfYearSin"] = np.sin(2 * np.pi * df["creationDate_stage"].dt.dayofyear / 365.0)
    df["dayOfYearCos"] = np.cos(2 * np.pi * df["creationDate_stage"].dt.dayofyear / 365.0)
    df["dayOfMonthSin"] = np.sin(2 * np.pi * df["creationDate_stage"].dt.day / 31.0)
    df["dayOfMonthCos"] = np.cos(2 * np.pi * df["creationDate_stage"].dt.day / 31.0)
    df["dayOfWeekSin"] = np.sin(2 * np.pi * df["creationDate_stage"].dt.weekday / 7.0)
    df["dayOfWeekCos"] = np.cos(2 * np.pi * df["creationDate_stage"].dt.weekday / 7.0)
    df["monthSin"] = np.sin(2 * np.pi * df["creationDate_stage"].dt.month / 12.0)
    df["monthCos"] = np.cos(2 * np.pi * df["creationDate_stage"].dt.month / 12.0)
    return df

vars_to_feature_engineer = ["creationTime"]  # 2

AV_consolidado = create_cyclic_features(AV_consolidado)

# Eliminación de columnas intermedias
AV_consolidado = AV_consolidado.drop(columns=["creationDate_stage", "creationTime_stage", *vars_to_feature_engineer])

# 3 vars_to_ohe
AV_consolidado_3_stage = AV_consolidado.copy()

# # Cambiar formato a "snake_case" sin tildes
# for column in vars_to_ohe:
#     AV_consolidado_3_stage[column] = (
#         AV_consolidado_3_stage[column]
#         .astype(str)
#         .apply(lambda x: unidecode(x).replace(" ", "_").lower())
#     )
#this is faster. from 5 1/2 minutes went to 1 1/2 minutes.
# Cambiar formato a "snake_case" sin tildes
for column in vars_to_ohe:
    AV_consolidado_3_stage[column] = (
        AV_consolidado_3_stage[column]
        .astype(str)
        .apply(unidecode)
        .str.replace(" ", "_", regex=False)
        .str.lower()
    )

In [49]:
AV_consolidado_3_stage.columns

Index(['debtorId', 'creditorCCI', 'channel', 'currency', 'creditorParticipant',
       'debtorParticipant', 'debtorTypeOfPerson', 'transactionType',
       'debtorIdCode', 'creditorCreditCard', 'responseCode', 'creditorId',
       'creditorIdCode', 'hourSin', 'hourCos', 'dayOfYearSin', 'dayOfYearCos',
       'dayOfMonthSin', 'dayOfMonthCos', 'dayOfWeekSin', 'dayOfWeekCos',
       'monthSin', 'monthCos'],
      dtype='object')

In [50]:
if PREDICTION_PIPELINE:
    ohe = OneHotEncoder(sparse=False, handle_unknown="ignore")
else:
    ohe = OneHotEncoder(sparse=False, handle_unknown="ignore", drop="first")
ohe.fit(AV_consolidado_3_stage[vars_to_ohe])


/home/cnvdba/miniconda3/envs/mvp1/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


OneHotEncoder(handle_unknown='ignore', sparse=False, sparse_output=False)

In [51]:
AV_consolidado_3_ohe_stage = pd.DataFrame(
    ohe.transform(AV_consolidado_3_stage[vars_to_ohe]),
    columns=ohe.get_feature_names_out(vars_to_ohe),
)

In [52]:
AV_consolidado = pd.concat(
    [AV_consolidado_3_stage.drop(vars_to_ohe, axis=1), AV_consolidado_3_ohe_stage],
    axis=1,
)

In [53]:
AV_consolidado_3_stage.head()

,debtorId,creditorCCI,channel,currency,creditorParticipant,debtorParticipant,debtorTypeOfPerson,transactionType,debtorIdCode,creditorCreditCard,...,hourSin,hourCos,dayOfYearSin,dayOfYearCos,dayOfMonthSin,dayOfMonthCos,dayOfWeekSin,dayOfWeekCos,monthSin,monthCos
0,99999999,01126300020026193410,invalid,soles,bbva,bcp,none,ordinary_transfer,2,None,...,0.0,1.0,0.409356,-0.912375,0.937752,0.347305,0.433884,-0.900969,1.224647e-16,-1.0
1,99999999,01113200020046033080,web,soles,bbva,bcp,none,ordinary_transfer,2,None,...,0.0,1.0,0.409356,-0.912375,0.937752,0.347305,0.433884,-0.900969,1.224647e-16,-1.0
2,71992204,00389801334501522345,web,soles,interbank,bcp,none,ordinary_transfer,2,None,...,0.0,1.0,0.409356,-0.912375,0.937752,0.347305,0.433884,-0.900969,1.224647e-16,-1.0
3,99999999,01102300010000606096,web,soles,bbva,bcp,none,ordinary_transfer,2,None,...,0.0,1.0,0.409356,-0.912375,0.937752,0.347305,0.433884,-0.900969,1.224647e-16,-1.0
4,99999999,00302301309958069994,web,soles,interbank,bcp,none,ordinary_transfer,2,None,...,0.0,1.0,0.409356,-0.912375,0.937752,0.347305,0.433884,-0.900969,1.224647e-16,-1.0


In [54]:
AV_consolidado[AV_consolidado_3_ohe_stage.columns] = AV_consolidado[
    AV_consolidado_3_ohe_stage.columns
].astype("uint8")
AV_consolidado_completed = pd.DataFrame()
AV_consolidado_completed = pd.concat(
    [AV_consolidado_completed, AV_consolidado], axis=0, ignore_index=True
)

In [55]:
AV_consolidado_original = pd.DataFrame()
AV_consolidado_original_stage = AV_consolidado.copy()
AV_consolidado_original = pd.concat(
    [AV_consolidado_original, AV_consolidado_original_stage],
    axis=0,
    ignore_index=True,
)
AV_consolidado_original_values_set = pd.DataFrame()
AV_consolidado_original_values_set = pd.concat(
    [AV_consolidado_original_values_set, AV_consolidado_original_values_set_stage],
    axis=0,
    ignore_index=True,
)


In [56]:
AV_consolidado_original_values_set["creationDate"].drop_duplicates().sort_values().reset_index(drop=True)

0     2024-06-06
1     2024-06-07
2     2024-06-08
3     2024-06-09
4     2024-06-10
         ...    
86    2024-08-31
87    2024-09-01
88    2024-09-02
89    2024-09-03
90    2024-09-04
Name: creationDate, Length: 91, dtype: object

In [57]:
# Fill nulls
AV_consolidado_completed.fillna(value=0, inplace=True)

# Set all binary to uint8 (except the ones that are uint8 already)
id_columns = [
    "debtorId",
    "debtorIdCode",
    "creditorCCI",
    "creditorCreditCard",
    "creditorId",
    "creditorIdCode",
]

excluded_columns = (
    [
        "hourSin",
        "hourCos",
        "dayOfYearSin",
        "dayOfYearCos",
        "dayOfMonthSin",
        "dayOfMonthCos",
        "dayOfWeekSin",
        "dayOfWeekCos",
        "monthSin",
        "monthCos",
    ]
    + AV_consolidado_completed.select_dtypes(include="uint8").columns.to_list()
    + id_columns
)



In [58]:
# Get a list of column names excluding 'excluded_columns' and id columns
binary_columns = list(
    set(AV_consolidado_completed.columns).difference(excluded_columns)
)


In [59]:
binary_columns

[]

In [60]:

# Convert the binary columns to uint8
AV_consolidado_completed[binary_columns] = AV_consolidado_completed[
    binary_columns
].astype("uint8")

In [27]:
# # Export AV_consolidado_completed
# output_file_path = "cce_ipf_message_feature_engineering.pickle"
# AV_consolidado_completed.to_pickle(output_file_path)

# # Export AV_consolidado_original
# output_file_path = "cce_ipf_message_original.pickle"
# AV_consolidado_original.to_pickle(output_file_path)

# # Export AV_consolidado_original_values_set
# output_file_path = "cce_ipf_message_original_values_set.pickle"
# AV_consolidado_original_values_set.to_pickle(output_file_path)
# print("end of execution")


In [26]:
AV_consolidado_completed.columns.tolist()

['debtorId',
 'creditorCCI',
 'debtorIdCode',
 'creditorCreditCard',
 'creditorId',
 'creditorIdCode',
 'hourSin',
 'hourCos',
 'dayOfYearSin',
 'dayOfYearCos',
 'dayOfMonthSin',
 'dayOfMonthCos',
 'dayOfWeekSin',
 'dayOfWeekCos',
 'monthSin',
 'monthCos',
 'debtorTypeOfPerson_none',
 'debtorParticipant_alfin_banco_s.a.',
 'debtorParticipant_banbif',
 'debtorParticipant_banco_de_la_nacion',
 'debtorParticipant_banco_falabella',
 'debtorParticipant_banco_pichincha',
 'debtorParticipant_banco_ripley',
 'debtorParticipant_bbva',
 'debtorParticipant_bcp',
 'debtorParticipant_caja_arequipa',
 'debtorParticipant_caja_cusco',
 'debtorParticipant_caja_ica',
 'debtorParticipant_caja_piura',
 'debtorParticipant_caja_trujillo',
 'debtorParticipant_crediscotia_financiera',
 'debtorParticipant_financiera_oh',
 'debtorParticipant_gnb',
 'debtorParticipant_interbank',
 'debtorParticipant_mi_banco',
 'debtorParticipant_scotiabank',
 'creditorParticipant_alfin_banco_s.a.',
 'creditorParticipant_banbif'

## FEATURE ENGINEERING RATIOS

In [27]:
def generate_combinations(input_list):
    """
    Generate all possible combinations of elements from the given input list.

    The function generates combinations by first including each element in a separate list,
    and then combining two elements at a time to form new combinations.

    Args:
        input_list (list): The list of elements from which to generate combinations.

    Returns:
        list: A list containing all possible combinations of elements.
    """
    
    output_list = []

    for i in range(len(input_list)):
        # Add each element in a list of 1 item
        output_list.append([input_list[i]])

        for j in range(i + 1, len(input_list)):
            # Add combinations of two elements
            output_list.append([input_list[i], input_list[j]])

    return output_list

def categorize_hour(hour):
    """
    Categorize the given hour into specific time intervals.

    Args:
        hour (int or str): The hour to be categorized. It can be an integer or a string representation of an integer.

    Returns:
        str: The category corresponding to the input hour.

    """
    # Zero-padding if needed
    hour = str(hour).zfill(6)  
    # Extract the hour part
    hour = int(hour[:2])  
    # Define time interval categories
    if hour >= 0 and hour < 3:
        return '00 to 03'
    elif hour >= 3 and hour < 6:
        return '03 to 06'
    elif hour >= 6 and hour < 9:
        return '06 to 09'
    elif hour >= 9 and hour < 12:
        return '09 to 12'
    elif hour >= 12 and hour < 15:
        return '12 to 15'
    elif hour >= 15 and hour < 18:
        return '15 to 18'
    elif hour >= 18 and hour < 21:
        return '18 to 21'
    else:  # hour >= 21 or hour < 24
        return '21 to 00'

In [28]:
# Define a list of columns to keep in the resulting DataFrame
lista = [
    "debtorId",
    "creditorCCI",
    "run_id",
    "creditorParticipant",
    "currency",
    "channel",
    "responseCode",
    "debtorParticipant",
    "creationDate",
    "creationTime",
    "time_interval",
    "Weekday",
]

# Generate all possible combinations of columns to be used for ratio calculations
output_list = generate_combinations(
    [
        "creditorParticipant",
        "currency",
        "channel",
        "responseCode",
        "debtorParticipant",
        "Weekday",
        "time_interval",
        "creditorCCI",
    ]
)

In [57]:
directory_path = ""
# Define the filenames for feature engineering and original values set DataFrames
file_name_feature_engineering = "cce_ipf_message_feature_engineering.pickle"
file_name_original_values_set = "cce_ipf_message_original_values_set.pickle"

# Build the file paths for the feature engineering and original values set DataFrames
file_path_fe = os.path.join(directory_path, file_name_feature_engineering)
file_path_o_vs = os.path.join(directory_path, file_name_original_values_set)

In [29]:
# Read the original values set DataFrame
# AV_o_vs = pd.read_pickle(file_path_o_vs)
AV_o_vs = AV_consolidado_original_values_set
# Filter and preprocess the original values set DataFrame
AV_o_vs = AV_o_vs[AV_o_vs["transactionType"] == "Ordinary Transfer"]
# AV_o_vs = AV_o_vs[AV_o_vs['debtorParticipant'].isin(code) | AV_o_vs['creditorParticipant'].isin(code)]
AV_o_vs["creationDate"] = pd.to_datetime(AV_o_vs["creationDate"])
df = AV_o_vs.copy()
df["Weekday"] = df["creationDate"].apply(lambda x: x.weekday()).astype(object)
df["time_interval"] = df["creationTime"].apply(categorize_hour)



/tmp/ipykernel_2403339/1182050474.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AV_o_vs["creationDate"] = pd.to_datetime(AV_o_vs["creationDate"])


In [30]:
len(AV_consolidado_original_values_set)

417526

In [31]:
len(AV_consolidado_original_values_set[AV_consolidado_original_values_set["transactionType"] != "Ordinary Transfer"])

19201

In [32]:
len(AV_o_vs)

398325

In [33]:
def Ratio(dd1,output_list,lista, days_back, max_run_id): #revisar la fecha
    """
    Calculate ratios based on cumulative counts for specified columns in the input DataFrame.

    The function takes a DataFrame `dd1` and calculates ratios for combinations of columns specified in `output_list`.
    The calculation is performed based on cumulative counts of each combination relative to a total count.

    Args:
        dd1 (pandas.DataFrame): The input DataFrame containing the data.
        output_list (list): A list of lists, where each inner list represents a combination of columns for which ratios
                            are to be calculated based on cumulative counts. The columns in each combination should be
                            present in the DataFrame `dd1`.
        lista (list): A list of column names from the DataFrame `dd1` that will be used to extract data.

    Returns:
        pandas.DataFrame: A DataFrame containing the calculated ratios for each combination specified in `output_list`.
    """
    
    creditor =["debtorId"]
    dd2_completed = pd.DataFrame()

    total_days = dd1.creationDate.drop_duplicates().sort_values().reset_index(drop=True)
    if PREDICTION_PIPELINE:
        training_days = [total_days.max()]
    else:
        training_days = total_days[total_days>= '2023-04-06'].reset_index(drop=True)

    print(training_days)
    for i, _day in enumerate(training_days):
        print(f"processing day {i+1} of {len(training_days)}: {_day}")
        dd2=dd1[(dd1.creationDate <= _day) & (dd1.creationDate >= _day - pd.Timedelta(days=days_back))][lista]\
            .sort_values(by=["debtorId","creationDate","creationTime"]).dropna().copy()
        dd2["count_cci"]=dd2.groupby(["debtorId"]).cumcount() + 1
        for i in output_list:
            dd2['_'.join(map(str, i))+"_cumcount"]=dd2.groupby(creditor+i).cumcount()
            dd2['_'.join(map(str, i))+"_ratio"]=dd2['_'.join(map(str, i))+"_cumcount"]/dd2["count_cci"]
        print("SIZE DD2 (before)", days_back, dd2.shape)
        if PREDICTION_PIPELINE:
            # dd2 = dd2[dd2.run_id == max(dd2.run_id)]
            dd2 = dd2[dd2.run_id == max_run_id]
        else:
            dd2 = dd2[dd2.creationDate == _day]
        print("SIZE DD2 (after)", days_back, dd2.shape)
        dd2_completed = pd.concat([dd2_completed, dd2], axis=0, ignore_index=False)
        
    return(dd2_completed.filter(like="ratio"))

In [34]:
df.creationDate.drop_duplicates().sort_values().reset_index(drop=True)

0    2024-05-30
1    2024-05-31
2    2024-06-01
3    2024-06-02
4    2024-06-03
        ...    
86   2024-08-24
87   2024-08-25
88   2024-08-26
89   2024-08-27
90   2024-08-28
Name: creationDate, Length: 91, dtype: datetime64[ns]

In [35]:
max_run_id

'20240828_0057'

In [36]:
# Calculate ratios based on cumulative counts for specified columns
ratios_df_7 = Ratio(df, output_list, lista, 7, max_run_id)
ratios_df_7.dropna(inplace=True)

ratios_df_90 = Ratio(df, output_list, lista, 90, max_run_id)
ratios_df_90.dropna(inplace=True)


[Timestamp('2024-08-28 00:00:00')]
processing day 1 of 1: 2024-08-28 00:00:00
SIZE DD2 (before) 7 (28675, 85)
SIZE DD2 (after) 7 (194, 85)
[Timestamp('2024-08-28 00:00:00')]
processing day 1 of 1: 2024-08-28 00:00:00
SIZE DD2 (before) 90 (398325, 85)
SIZE DD2 (after) 90 (194, 85)


In [37]:
# Read the feature engineering DataFrame
# AV_fe = pd.read_pickle(file_path_fe)
AV_fe = AV_consolidado_completed
AV_fe = AV_fe.drop(
    ["debtorIdCode", "creditorCreditCard", "creditorId", "creditorIdCode"], axis=1
)

# Merge the calculated ratios DataFrame and the feature engineering DataFrame
final_7 = pd.merge(ratios_df_7, AV_fe, left_index=True, right_index=True)
final_90 = pd.merge(ratios_df_90, AV_fe, left_index=True, right_index=True)

print("SHAPE 7 days", final_7.shape)
print("SHAPE 90 days", final_90.shape)

# Save the final DataFrame to a pickle file
# final_7.to_pickle(os.path.join(directory_path, "df_output_7.pickle"))
# final_90.to_pickle(os.path.join(directory_path, "df_output_90.pickle"))

SHAPE 7 days (194, 103)
SHAPE 90 days (194, 103)


## TRAIN MODEL

In [ ]:
RS = 12494328
train_data_file_name = "df_output.pickle"
AV_train = pd.read_pickle(train_data_file_name)
print("nasss", AV_train.isna().sum() / AV_train.shape[0])
print(AV_train)
# Explore train data
print(f"AV_train shape: {AV_train.shape}")
print(f"AV_train columns: {AV_train.columns}")
print(f"creditorCCI únicos: {AV_train.creditorCCI.nunique()}")
print("Top 10 creditor CCI con más operaciones AV:")
print(AV_train.creditorCCI.value_counts().sort_values(ascending=False).head(10))
AV_train_nunique = AV_train.nunique()
cols_to_drop_unique_value = AV_train_nunique[AV_train_nunique == 1].index.to_list()
# drop columns with unique value
if len(cols_to_drop_unique_value) > 0:
    AV_train.drop(cols_to_drop_unique_value, axis=1, inplace=True)
print(AV_train.shape)
AV_train.dropna(inplace=True)
print(AV_train.shape)

args_5 = {"random_state": RS, "contamination": 0.0196}
id_cols = ["debtorId", "creditorCCI"]



In [40]:
variables = [
    "creditorParticipant_ratio",
    "creditorParticipant_currency_ratio",
    "creditorParticipant_channel_ratio",
    "creditorParticipant_responseCode_ratio",
    "creditorParticipant_debtorParticipant_ratio",
    "creditorParticipant_Weekday_ratio",
    "creditorParticipant_time_interval_ratio",
    "creditorParticipant_creditorCCI_ratio",
    "currency_ratio",
    "currency_channel_ratio",
    "currency_responseCode_ratio",
    "currency_debtorParticipant_ratio",
    "currency_Weekday_ratio",
    "currency_time_interval_ratio",
    "currency_creditorCCI_ratio",
    "channel_ratio",
    "channel_responseCode_ratio",
    "channel_debtorParticipant_ratio",
    "channel_Weekday_ratio",
    "channel_time_interval_ratio",
    "channel_creditorCCI_ratio",
    "responseCode_ratio",
    "responseCode_debtorParticipant_ratio",
    "responseCode_Weekday_ratio",
    "responseCode_time_interval_ratio",
    "responseCode_creditorCCI_ratio",
    "debtorParticipant_ratio",
    "debtorParticipant_Weekday_ratio",
    "debtorParticipant_time_interval_ratio",
    "debtorParticipant_creditorCCI_ratio",
    "Weekday_ratio",
    "Weekday_time_interval_ratio",
    "Weekday_creditorCCI_ratio",
    "time_interval_ratio",
    "time_interval_creditorCCI_ratio",
    "creditorCCI_ratio",
    "debtorId",
    "creditorCCI",
    "hourSin",
    "hourCos",
    "dayOfYearSin",
    "dayOfYearCos",
    "dayOfMonthSin",
    "dayOfMonthCos",
    "dayOfWeekSin",
    "dayOfWeekCos",
    "debtorParticipant_bcp",
    "debtorParticipant_interbank",
    "debtorParticipant_citibank",
    "debtorParticipant_scotiabank",
    "debtorParticipant_bbva",
    "debtorParticipant_banco_de_la_nacion",
    "debtorParticipant_comercio",
    "debtorParticipant_banco_pichincha",
    "debtorParticipant_banbif",
    "debtorParticipant_crediscotia_financiera",
    "debtorParticipant_mi_banco",
    "debtorParticipant_gnb",
    "debtorParticipant_banco_falabella",
    "debtorParticipant_banco_ripley",
    "debtorParticipant_alfin_banco_s.a.",
    "debtorParticipant_financiera_oh",
    "debtorParticipant_financiera_efectiva",
    "debtorParticipant_caja_piura",
    "debtorParticipant_caja_trujillo",
    "debtorParticipant_caja_arequipa",
    "debtorParticipant_caja_sullana",
    "debtorParticipant_caja_cusco",
    "debtorParticipant_caja_huancayo",
    "debtorParticipant_caja_ica",
    "debtorParticipant_invalid",
    "creditorParticipant_bcp",
    "creditorParticipant_interbank",
    "creditorParticipant_citibank",
    "creditorParticipant_scotiabank",
    "creditorParticipant_bbva",
    "creditorParticipant_banco_de_la_nacion",
    "creditorParticipant_comercio",
    "creditorParticipant_banco_pichincha",
    "creditorParticipant_banbif",
    "creditorParticipant_crediscotia_financiera",
    "creditorParticipant_mi_banco",
    "creditorParticipant_gnb",
    "creditorParticipant_banco_falabella",
    "creditorParticipant_banco_ripley",
    "creditorParticipant_alfin_banco_s.a.",
    "creditorParticipant_financiera_oh",
    "creditorParticipant_financiera_efectiva",
    "creditorParticipant_caja_piura",
    "creditorParticipant_caja_trujillo",
    "creditorParticipant_caja_arequipa",
    "creditorParticipant_caja_sullana",
    "creditorParticipant_caja_cusco",
    "creditorParticipant_caja_huancayo",
    "creditorParticipant_caja_ica",
    "creditorParticipant_invalid",
    "currency_soles",
    "channel_banca_movil",
    "channel_invalid",
    "channel_web",
    "responseCode_rejected",
]

for col in variables:
    if col not in AV_train.columns:
        AV_train[col] = 0

AV_train = AV_train[variables]
print("variables train", AV_train.columns.tolist())

NameError: name 'AV_train' is not defined

In [ ]:
# model function
def fit_isolation_forest(input_data, args=None):
    rs = args["random_state"]
    c = args["contamination"]
   
    model = IsolationForest(random_state=rs, contamination=c, n_jobs=-1)
    model.fit(input_data)

    return model


In [ ]:
if_model_5 = fit_isolation_forest(AV_train.drop(id_cols, axis=1), args_5)
output_directory_path = ""
file_path = os.path.join(output_directory_path, "mvp_1_model_90_days.pickle")
with open(file_path, "wb") as handle:
    pickle.dump(if_model_5, handle, protocol=pickle.HIGHEST_PROTOCOL)

## PREDICT MODEL

In [38]:
def load_model(file_path, file_name):
    full_file_path = os.path.join(file_path, file_name)
    with open(full_file_path, 'rb') as handle:
        loaded_model = pickle.load(handle)
    return loaded_model

In [40]:
# Path and file inputs
# trained_models_path = '/home/cnvdba/cce/src/Models/Train'
trained_models_path = ''
trained_model_7_file_name = 'mvp_1_model_7_opt_200K.pickle' # 'mvp_1_model.pickle' 
trained_model_90_file_name = 'mvp_1_model_90_opt_200K.pickle'

transformed_data_path = ''
transformed_data_file_path_7 = os.path.join(transformed_data_path, 'df_output_7.pickle')
transformed_data_file_path_90 = os.path.join(transformed_data_path, 'df_output_90.pickle')
original_values_set_data_file_path = os.path.join(transformed_data_path, 'cce_ipf_message_original_values_set.pickle')

# Load data
model_7 = load_model(trained_models_path, trained_model_7_file_name)
model_90 = load_model(trained_models_path, trained_model_90_file_name)
original_values_set = AV_consolidado_original_values_set #pd.read_pickle(original_values_set_data_file_path)

# transformed data
AV_7 = final_7 #pd.read_pickle(transformed_data_file_path_7)
AV_90 = final_90 #pd.read_pickle(transformed_data_file_path_90)

In [43]:
# variables=['creditorParticipant_ratio', 'creditorParticipant_currency_ratio',
#    'creditorParticipant_channel_ratio',
#    'creditorParticipant_responseCode_ratio',
#    'creditorParticipant_debtorParticipant_ratio',
#    'creditorParticipant_Weekday_ratio',
#    'creditorParticipant_time_interval_ratio',
#    'creditorParticipant_creditorCCI_ratio', 'currency_ratio',
#    'currency_channel_ratio', 'currency_responseCode_ratio',
#    'currency_debtorParticipant_ratio', 'currency_Weekday_ratio',
#    'currency_time_interval_ratio', 'currency_creditorCCI_ratio',
#    'channel_ratio', 'channel_responseCode_ratio',
#    'channel_debtorParticipant_ratio', 'channel_Weekday_ratio',
#    'channel_time_interval_ratio', 'channel_creditorCCI_ratio',
#    'responseCode_ratio', 'responseCode_debtorParticipant_ratio',
#    'responseCode_Weekday_ratio', 'responseCode_time_interval_ratio',
#    'responseCode_creditorCCI_ratio', 'debtorParticipant_ratio',
#    'debtorParticipant_Weekday_ratio',
#    'debtorParticipant_time_interval_ratio',
#    'debtorParticipant_creditorCCI_ratio', 'Weekday_ratio',
#    'Weekday_time_interval_ratio', 'Weekday_creditorCCI_ratio',
#    'time_interval_ratio', 'time_interval_creditorCCI_ratio',
#    'creditorCCI_ratio', 'hourSin', 'hourCos', 'dayOfYearSin',
#    'dayOfYearCos', 'dayOfMonthSin', 'dayOfMonthCos', 'dayOfWeekSin',
#    'dayOfWeekCos', 'monthSin', 'monthCos', 'debtorTypeOfPerson_n',
#    'debtorParticipant_banco_falabella',
#    'debtorParticipant_banco_pichincha', 'debtorParticipant_bbva',
#    'debtorParticipant_bcp', 'debtorParticipant_caja_arequipa',
#    'debtorParticipant_caja_cusco', 'debtorParticipant_caja_ica',
#    'debtorParticipant_caja_piura', 'debtorParticipant_caja_sullana',
#    'debtorParticipant_caja_trujillo', 'debtorParticipant_comercio',
#    'debtorParticipant_crediscotia_financiera', 'debtorParticipant_gnb',
#    'debtorParticipant_interbank', 'debtorParticipant_mi_banco',
#    'debtorParticipant_scotiabank', 'creditorParticipant_banbif',
#    'creditorParticipant_banco_de_la_nacion',
#    'creditorParticipant_banco_falabella',
#    'creditorParticipant_banco_pichincha', 'creditorParticipant_bbva',
#    'creditorParticipant_bcp', 'creditorParticipant_caja_arequipa',
#    'creditorParticipant_caja_cusco', 'creditorParticipant_caja_ica',
#    'creditorParticipant_caja_piura', 'creditorParticipant_caja_sullana',
#    'creditorParticipant_caja_trujillo', 'creditorParticipant_citibank',
#    'creditorParticipant_comercio',
#    'creditorParticipant_crediscotia_financiera', 'creditorParticipant_gnb',
#    'creditorParticipant_interbank', 'creditorParticipant_mi_banco',
#    'creditorParticipant_scotiabank', 'currency_soles',
#    'channel_banca_movil', 'channel_invalid', 'channel_web',
#    'responseCode_rejected', 'debtorParticipant_financiera_oh',
#    'creditorParticipant_financiera_oh', 'debtorParticipant_banbif']
# variables = ['creditorParticipant_ratio', 'creditorParticipant_currency_ratio', 'creditorParticipant_channel_ratio', 'creditorParticipant_responseCode_ratio', 'creditorParticipant_debtorParticipant_ratio', 'creditorParticipant_Weekday_ratio', 'creditorParticipant_time_interval_ratio', 'creditorParticipant_creditorCCI_ratio', 'currency_ratio', 'currency_channel_ratio', 'currency_responseCode_ratio', 'currency_debtorParticipant_ratio', 'currency_Weekday_ratio', 'currency_time_interval_ratio', 'currency_creditorCCI_ratio', 'channel_ratio', 'channel_responseCode_ratio', 'channel_debtorParticipant_ratio', 'channel_Weekday_ratio', 'channel_time_interval_ratio', 'channel_creditorCCI_ratio', 'responseCode_ratio', 'responseCode_debtorParticipant_ratio', 'responseCode_Weekday_ratio', 'responseCode_time_interval_ratio', 'responseCode_creditorCCI_ratio', 'debtorParticipant_ratio', 'debtorParticipant_Weekday_ratio', 'debtorParticipant_time_interval_ratio', 'debtorParticipant_creditorCCI_ratio', 'Weekday_ratio', 'Weekday_time_interval_ratio', 'Weekday_creditorCCI_ratio', 'time_interval_ratio', 'time_interval_creditorCCI_ratio', 'creditorCCI_ratio', 'hourSin', 'hourCos', 'dayOfYearSin', 'dayOfYearCos', 'dayOfMonthSin', 'dayOfMonthCos', 'dayOfWeekSin', 'dayOfWeekCos', 'debtorParticipant_banbif', 'debtorParticipant_banco_de_la_nacion', 'debtorParticipant_banco_falabella', 'debtorParticipant_banco_pichincha', 'debtorParticipant_banco_ripley', 'debtorParticipant_bbva', 'debtorParticipant_bcp', 'debtorParticipant_caja_arequipa', 'debtorParticipant_caja_cusco', 'debtorParticipant_caja_huancayo', 'debtorParticipant_caja_ica', 'debtorParticipant_caja_piura', 'debtorParticipant_caja_sullana', 'debtorParticipant_caja_trujillo', 'debtorParticipant_comercio', 'debtorParticipant_crediscotia_financiera', 'debtorParticipant_financiera_efectiva', 'debtorParticipant_financiera_oh', 'debtorParticipant_gnb', 'debtorParticipant_interbank', 'debtorParticipant_invalid', 'debtorParticipant_mi_banco', 'debtorParticipant_scotiabank', 'creditorParticipant_banbif', 'creditorParticipant_banco_de_la_nacion', 'creditorParticipant_banco_falabella', 'creditorParticipant_banco_pichincha', 'creditorParticipant_banco_ripley', 'creditorParticipant_bbva', 'creditorParticipant_bcp', 'creditorParticipant_caja_arequipa', 'creditorParticipant_caja_cusco', 'creditorParticipant_caja_huancayo', 'creditorParticipant_caja_ica', 'creditorParticipant_caja_piura', 'creditorParticipant_caja_sullana', 'creditorParticipant_caja_trujillo', 'creditorParticipant_citibank', 'creditorParticipant_comercio', 'creditorParticipant_crediscotia_financiera', 'creditorParticipant_financiera_efectiva', 'creditorParticipant_financiera_oh', 'creditorParticipant_gnb', 'creditorParticipant_interbank', 'creditorParticipant_invalid', 'creditorParticipant_mi_banco', 'creditorParticipant_scotiabank', 'currency_soles', 'channel_banca_movil', 'channel_invalid', 'channel_web', 'responseCode_rejected']
variables = ['creditorParticipant_ratio', 'creditorParticipant_currency_ratio', 'creditorParticipant_channel_ratio', 'creditorParticipant_responseCode_ratio', 'creditorParticipant_debtorParticipant_ratio', 'creditorParticipant_Weekday_ratio', 'creditorParticipant_time_interval_ratio', 'creditorParticipant_creditorCCI_ratio', 'currency_ratio', 'currency_channel_ratio', 'currency_responseCode_ratio', 'currency_debtorParticipant_ratio', 'currency_Weekday_ratio', 'currency_time_interval_ratio', 'currency_creditorCCI_ratio', 'channel_ratio', 'channel_responseCode_ratio', 'channel_debtorParticipant_ratio', 'channel_Weekday_ratio', 'channel_time_interval_ratio', 'channel_creditorCCI_ratio', 'responseCode_ratio', 'responseCode_debtorParticipant_ratio', 'responseCode_Weekday_ratio', 'responseCode_time_interval_ratio', 'responseCode_creditorCCI_ratio', 'debtorParticipant_ratio', 'debtorParticipant_Weekday_ratio', 'debtorParticipant_time_interval_ratio', 'debtorParticipant_creditorCCI_ratio', 'Weekday_ratio', 'Weekday_time_interval_ratio', 'Weekday_creditorCCI_ratio', 'time_interval_ratio', 'time_interval_creditorCCI_ratio', 'creditorCCI_ratio', 'hourSin', 'hourCos', 'dayOfYearSin', 'dayOfYearCos', 'dayOfMonthSin', 'dayOfMonthCos', 'dayOfWeekSin', 'dayOfWeekCos','debtorParticipant_bcp','debtorParticipant_interbank','debtorParticipant_citibank','debtorParticipant_scotiabank','debtorParticipant_bbva','debtorParticipant_banco_de_la_nacion','debtorParticipant_comercio','debtorParticipant_banco_pichincha','debtorParticipant_banbif','debtorParticipant_crediscotia_financiera','debtorParticipant_mi_banco','debtorParticipant_gnb','debtorParticipant_banco_falabella','debtorParticipant_banco_ripley','debtorParticipant_alfin_banco_s.a.','debtorParticipant_financiera_oh','debtorParticipant_financiera_efectiva','debtorParticipant_caja_piura','debtorParticipant_caja_trujillo','debtorParticipant_caja_arequipa','debtorParticipant_caja_sullana','debtorParticipant_caja_cusco','debtorParticipant_caja_huancayo','debtorParticipant_caja_ica','debtorParticipant_invalid','creditorParticipant_bcp','creditorParticipant_interbank','creditorParticipant_citibank','creditorParticipant_scotiabank','creditorParticipant_bbva','creditorParticipant_banco_de_la_nacion','creditorParticipant_comercio','creditorParticipant_banco_pichincha','creditorParticipant_banbif','creditorParticipant_crediscotia_financiera','creditorParticipant_mi_banco','creditorParticipant_gnb','creditorParticipant_banco_falabella','creditorParticipant_banco_ripley','creditorParticipant_alfin_banco_s.a.','creditorParticipant_financiera_oh','creditorParticipant_financiera_efectiva','creditorParticipant_caja_piura','creditorParticipant_caja_trujillo','creditorParticipant_caja_arequipa','creditorParticipant_caja_sullana','creditorParticipant_caja_cusco','creditorParticipant_caja_huancayo','creditorParticipant_caja_ica','creditorParticipant_invalid', 'currency_soles', 'channel_banca_movil', 'channel_invalid', 'channel_web', 'responseCode_rejected']
# Crear columnas si faltaran
for col in variables:
    if col not in AV_7.columns:
        AV_7[col] = 0
for col in variables:
    if col not in AV_90.columns:
        AV_90[col] = 0



In [58]:
def predict_isolation_forest(input_data, model):
    print(input_data.columns.tolist())
    y_pred = model.predict(input_data)
    y_pred = [1 if pred == -1 else 0 for pred in y_pred]
    y_pred_series = pd.Series(data=y_pred, index=input_data.index).astype("uint8")
    return y_pred_series

def score_isolation_forest(input_data, model):
    print(input_data.columns.tolist())
    y_pred = model.decision_function(input_data)
    min_value = -0.11774102106942208
    max_value = 0.1602303765911105
    # norm_y_pred = (y_pred - min_value) / (max_value - min_value) * 99
    norm_y_pred = (max_value - y_pred) / (max_value - min_value) * 99
    # norm_y_pred = max(0, min(norm_y_pred, 99))
    # y_pred = [1 if pred == -1 else 0 for pred in y_pred]
    norm_y_pred = norm_y_pred.clip(min=0.0, max=99.0)
    # y_pred_series = pd.Series(data=y_pred, index=input_data.index).astype("float64")
    y_pred_series = pd.Series(data=norm_y_pred, index=input_data.index).astype("float64")
    return y_pred_series

In [45]:
AV_7 = AV_7[variables].copy()
AV_90 = AV_90[variables].copy()

# print(AV_7.columns.tolist())
# print(AV_90.columns.tolist())
y_pred_7 = predict_isolation_forest(AV_7, model_7)
y_pred_90 = predict_isolation_forest(AV_7, model_90)
    

['creditorParticipant_ratio', 'creditorParticipant_currency_ratio', 'creditorParticipant_channel_ratio', 'creditorParticipant_responseCode_ratio', 'creditorParticipant_debtorParticipant_ratio', 'creditorParticipant_Weekday_ratio', 'creditorParticipant_time_interval_ratio', 'creditorParticipant_creditorCCI_ratio', 'currency_ratio', 'currency_channel_ratio', 'currency_responseCode_ratio', 'currency_debtorParticipant_ratio', 'currency_Weekday_ratio', 'currency_time_interval_ratio', 'currency_creditorCCI_ratio', 'channel_ratio', 'channel_responseCode_ratio', 'channel_debtorParticipant_ratio', 'channel_Weekday_ratio', 'channel_time_interval_ratio', 'channel_creditorCCI_ratio', 'responseCode_ratio', 'responseCode_debtorParticipant_ratio', 'responseCode_Weekday_ratio', 'responseCode_time_interval_ratio', 'responseCode_creditorCCI_ratio', 'debtorParticipant_ratio', 'debtorParticipant_Weekday_ratio', 'debtorParticipant_time_interval_ratio', 'debtorParticipant_creditorCCI_ratio', 'Weekday_ratio'

In [59]:
y_score_7 = score_isolation_forest(AV_7, model_7)
y_score_90 = score_isolation_forest(AV_7, model_90)

['creditorParticipant_ratio', 'creditorParticipant_currency_ratio', 'creditorParticipant_channel_ratio', 'creditorParticipant_responseCode_ratio', 'creditorParticipant_debtorParticipant_ratio', 'creditorParticipant_Weekday_ratio', 'creditorParticipant_time_interval_ratio', 'creditorParticipant_creditorCCI_ratio', 'currency_ratio', 'currency_channel_ratio', 'currency_responseCode_ratio', 'currency_debtorParticipant_ratio', 'currency_Weekday_ratio', 'currency_time_interval_ratio', 'currency_creditorCCI_ratio', 'channel_ratio', 'channel_responseCode_ratio', 'channel_debtorParticipant_ratio', 'channel_Weekday_ratio', 'channel_time_interval_ratio', 'channel_creditorCCI_ratio', 'responseCode_ratio', 'responseCode_debtorParticipant_ratio', 'responseCode_Weekday_ratio', 'responseCode_time_interval_ratio', 'responseCode_creditorCCI_ratio', 'debtorParticipant_ratio', 'debtorParticipant_Weekday_ratio', 'debtorParticipant_time_interval_ratio', 'debtorParticipant_creditorCCI_ratio', 'Weekday_ratio'

In [52]:
y_score_7.min(), 


-0.025661208596230667

In [46]:
y_pred_7.shape, y_pred_90.shape

((194,), (194,))

In [79]:

# Path to the directory to sAV_7e the output of prediction
y_pred_7.to_pickle("prediction_model_7.pickle")
y_pred_90.to_pickle("prediction_model_90.pickle")

# Dataframe fraud_model_predict (features id + descriptive + rate + prediction)
id_descriptive_features_list = \
['pk', 'debtorId', 'creditorCCI', 'creditorId', 'creditorIdCode', 'message_id', 'trace', 'instruction_id', 'run_id',
    'creationDate', 'creationTime', 'channel', 'currency', 'creditorParticipant', 'debtorParticipant',
    'debtorTypeOfPerson', 'transactionType', 'debtorIdCode', 'reasonCode', 'responseCode', ]
rate_features_list_7 = AV_7.filter(like="_ratio").columns.to_list()
rate_features_list_90 = AV_90.filter(like="_ratio").columns.to_list()

df_fraud_model_predict_7 = original_values_set[id_descriptive_features_list]\
    .merge(AV_7[rate_features_list_7], how="inner", left_index=True, right_index=True)
df_fraud_model_predict_7 = df_fraud_model_predict_7\
    .merge(y_pred_7.rename("prediction"), how="inner", left_index=True, right_index=True)

df_fraud_model_predict_7 = df_fraud_model_predict_7\
    .merge(y_score_7.rename("score"), how="inner", left_index=True, right_index=True)
df_fraud_model_predict_7.columns=["pk","debtor_id","creditor_cci","creditor_id","creditor_id_code","message_id","trace","instruction_id","run_id","creation_date","creation_time","channel","currency","creditor_participant","debtor_participant","debtor_type_of_person","transaction_type","debtor_id_code","reason_code","response_code","creditor_participant_ratio","creditor_participant_currency_ratio","creditor_participant_channel_ratio","creditor_participant_response_code_ratio","creditor_participant_debtor_participant_ratio","creditor_participant_weekday_ratio","creditor_participant_time_interval_ratio","creditor_participant_creditor_cci_ratio","currency_ratio","currency_channel_ratio","currency_response_code_ratio","currency_debtor_participant_ratio","currency_weekday_ratio","currency_time_interval_ratio","currency_creditor_cci_ratio","channel_ratio","channel_response_code_ratio","channel_debtor_participant_ratio","channel_weekday_ratio","channel_time_interval_ratio","channel_creditor_cci_ratio","response_code_ratio","response_code_debtor_participant_ratio","response_code_weekday_ratio","response_code_time_interval_ratio","response_code_creditor_cci_ratio","debtor_participant_ratio","debtor_participant_weekday_ratio","debtor_participant_time_interval_ratio","debtor_participant_creditor_cci_ratio", "weekday_ratio","weekday_time_interval_ratio","weekday_creditor_cci_ratio","time_interval_ratio","time_interval_creditor_cci_ratio","creditor_cci_ratio","prediction", "score"]    # Load predictions to table
#df_fraud_model_predict_7["pk"]=df_fraud_model_predict_7["pk"]+"_l1"

df_fraud_model_predict_90 = original_values_set[id_descriptive_features_list]\
    .merge(AV_90[rate_features_list_90], how="inner", left_index=True, right_index=True)
df_fraud_model_predict_90 = df_fraud_model_predict_90\
    .merge(y_pred_90.rename("prediction"), how="inner", left_index=True, right_index=True)
df_fraud_model_predict_90.columns=["pk","debtor_id","creditor_cci","creditor_id","creditor_id_code","message_id","trace","instruction_id","run_id","creation_date","creation_time","channel","currency","creditor_participant","debtor_participant","debtor_type_of_person","transaction_type","debtor_id_code","reason_code","response_code","creditor_participant_ratio","creditor_participant_currency_ratio","creditor_participant_channel_ratio","creditor_participant_response_code_ratio","creditor_participant_debtor_participant_ratio","creditor_participant_weekday_ratio","creditor_participant_time_interval_ratio","creditor_participant_creditor_cci_ratio","currency_ratio","currency_channel_ratio","currency_response_code_ratio","currency_debtor_participant_ratio","currency_weekday_ratio","currency_time_interval_ratio","currency_creditor_cci_ratio","channel_ratio","channel_response_code_ratio","channel_debtor_participant_ratio","channel_weekday_ratio","channel_time_interval_ratio","channel_creditor_cci_ratio","response_code_ratio","response_code_debtor_participant_ratio","response_code_weekday_ratio","response_code_time_interval_ratio","response_code_creditor_cci_ratio","debtor_participant_ratio","debtor_participant_weekday_ratio","debtor_participant_time_interval_ratio","debtor_participant_creditor_cci_ratio", "weekday_ratio","weekday_time_interval_ratio","weekday_creditor_cci_ratio","time_interval_ratio","time_interval_creditor_cci_ratio","creditor_cci_ratio","prediction"]    # Load predictions to table
#df_fraud_model_predict_90["pk"]=df_fraud_model_predict_90["pk"]+"_l1"
# model_version = 'MOD_OLD_7'
# load_predictions_to_database(df_fraud_model_predict_7, 'MOD_7_280824_v1')
# load_predictions_to_database(df_fraud_model_predict_90, 'MOD_90_280824_v1')


# # Update process status
# # run_id = df_fraud_model_predict_7.run_id.max()
# run_id = max_run_id
# update_process_status_to_database(run_id,2,"run_process_end_datetime")

In [80]:
df_fraud_model_predict_7.head()

,pk,debtor_id,creditor_cci,creditor_id,creditor_id_code,message_id,trace,instruction_id,run_id,creation_date,...,debtor_participant_time_interval_ratio,debtor_participant_creditor_cci_ratio,weekday_ratio,weekday_time_interval_ratio,weekday_creditor_cci_ratio,time_interval_ratio,time_interval_creditor_cci_ratio,creditor_cci_ratio,prediction,score
390726,3305251443_003347_320_0002_0018_20240828,75279319,00219110155250708851,75279319,2,3305251443,003347,2024082804412100188152003347,20240828_0057,2024-08-28,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,42.513859
390727,3305251650_718217_320_0003_0038_20240828,41018471,00336701319200535004,41018471,2,3305251650,718217,2024082804410200388115718217,20240828_0057,2024-08-28,...,0.000000,0.833333,0.000000,0.000000,0.000000,0.000000,0.000000,0.833333,0,34.965505
390728,3305251670_033331_320_0002_0018_20240828,71106676,00235017552542503970,71106676,2,3305251670,033331,2024082804413000188191033331,20240828_0057,2024-08-28,...,0.090909,0.909091,0.272727,0.000000,0.272727,0.090909,0.090909,0.909091,0,27.143799
390729,3305252105_126540_320_0054_0002_20240828,74645897,05410382103144792875,None,None,3305252105,126540,2024082804411100028152126540,20240828_0057,2024-08-28,...,0.100000,0.400000,0.000000,0.000000,0.000000,0.200000,0.100000,0.400000,0,41.524242
390730,3305252114_126541_320_0003_0002_20240828,71882483,00389801316240114042,71882483,2,3305252114,126541,2024082804411200028152126541,20240828_0057,2024-08-28,...,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0,48.134314


In [42]:
def predict_isolation_forest(input_data, model):
    print(input_data.columns.tolist())
    y_pred = model.predict(input_data)
    y_pred = [1 if pred == -1 else 0 for pred in y_pred]
    y_pred_series = pd.Series(data=y_pred, index=input_data.index).astype("uint8")
    return y_pred_series


In [170]:
AV = AV[variables].copy()
print(AV.columns.tolist())
y_pred_5 = predict_isolation_forest(AV, model5)
    

['creditorParticipant_ratio', 'creditorParticipant_currency_ratio', 'creditorParticipant_channel_ratio', 'creditorParticipant_responseCode_ratio', 'creditorParticipant_debtorParticipant_ratio', 'creditorParticipant_Weekday_ratio', 'creditorParticipant_time_interval_ratio', 'creditorParticipant_creditorCCI_ratio', 'currency_ratio', 'currency_channel_ratio', 'currency_responseCode_ratio', 'currency_debtorParticipant_ratio', 'currency_Weekday_ratio', 'currency_time_interval_ratio', 'currency_creditorCCI_ratio', 'channel_ratio', 'channel_responseCode_ratio', 'channel_debtorParticipant_ratio', 'channel_Weekday_ratio', 'channel_time_interval_ratio', 'channel_creditorCCI_ratio', 'responseCode_ratio', 'responseCode_debtorParticipant_ratio', 'responseCode_Weekday_ratio', 'responseCode_time_interval_ratio', 'responseCode_creditorCCI_ratio', 'debtorParticipant_ratio', 'debtorParticipant_Weekday_ratio', 'debtorParticipant_time_interval_ratio', 'debtorParticipant_creditorCCI_ratio', 'Weekday_ratio'

In [172]:
y_pred_5.shape

(5847,)

In [177]:

# Path to the directory to save the output of prediction
y_pred_5.to_pickle("prediction_model_90_7M.pickle")

# Dataframe fraud_model_predict (features id + descriptive + rate + prediction)
id_descriptive_features_list = \
['pk', 'debtorId', 'creditorCCI', 'creditorId', 'creditorIdCode', 'message_id', 'trace', 'instruction_id', 'run_id',
    'creationDate', 'creationTime', 'channel', 'currency', 'creditorParticipant', 'debtorParticipant',
    'debtorTypeOfPerson', 'transactionType', 'debtorIdCode', 'reasonCode', 'responseCode', ]
rate_features_list = AV.filter(like="_ratio").columns.to_list()

df_fraud_model_predict = original_values_set[id_descriptive_features_list]\
    .merge(AV[rate_features_list], how="inner", left_index=True, right_index=True)
df_fraud_model_predict = df_fraud_model_predict\
    .merge(y_pred_5.rename("prediction"), how="inner", left_index=True, right_index=True)
df_fraud_model_predict.columns=["pk","debtor_id","creditor_cci","creditor_id","creditor_id_code","message_id","trace","instruction_id","run_id","creation_date","creation_time","channel","currency","creditor_participant","debtor_participant","debtor_type_of_person","transaction_type","debtor_id_code","reason_code","response_code","creditor_participant_ratio","creditor_participant_currency_ratio","creditor_participant_channel_ratio","creditor_participant_response_code_ratio","creditor_participant_debtor_participant_ratio","creditor_participant_weekday_ratio","creditor_participant_time_interval_ratio","creditor_participant_creditor_cci_ratio","currency_ratio","currency_channel_ratio","currency_response_code_ratio","currency_debtor_participant_ratio","currency_weekday_ratio","currency_time_interval_ratio","currency_creditor_cci_ratio","channel_ratio","channel_response_code_ratio","channel_debtor_participant_ratio","channel_weekday_ratio","channel_time_interval_ratio","channel_creditor_cci_ratio","response_code_ratio","response_code_debtor_participant_ratio","response_code_weekday_ratio","response_code_time_interval_ratio","response_code_creditor_cci_ratio","debtor_participant_ratio","debtor_participant_weekday_ratio","debtor_participant_time_interval_ratio","debtor_participant_creditor_cci_ratio", "weekday_ratio","weekday_time_interval_ratio","weekday_creditor_cci_ratio","time_interval_ratio","time_interval_creditor_cci_ratio","creditor_cci_ratio","prediction"]    # Load predictions to table
#df_fraud_model_predict["pk"]=df_fraud_model_predict["pk"]+"_l1"
# load_predictions_to_database(df_fraud_model_predict)

# Update process status
# run_id = df_fraud_model_predict.run_id.max()

# update_process_status_to_database(run_id,2,"run_process_end_datetime")